In [1]:
# Standard libraries
import pandas as pd
import numpy as np
from scipy import stats
# Visualization
import matplotlib.pyplot as plt
import wrds
# import os 
# os.system("pip install wrds") # TODO: Probably put this in utils.py

<b>TODO LIST:</b>
1. Parse audit analytics data for year and month of date of disclosure (possibly include date of breach) - Might have to select a couple of firms that we want to analyze?
    * Use year, month, and ticker to find stock price of company for that month (in WorldScope database)
    * Find stock price 1 to 12 months from that date (WorldScope). Calculate gain/loss in stock price from original. Potentially even take stock price the month before the breach?
2. Perform statistical tests + data visualization/analysis. Some examples:
    * Regression on multiple variables to predict stock price gain/loss over x months 
    * Scatter plot over time (ex: 2000-2021) of stock price gains/losses
    * Scatter plot over time after date of disclosure/breach of stock price gains/losses
    * Use Regression to see if type of breach matters for stock price
    * Regression to predict records lost?
    * Create decision tree to see type of breach

<b>Problems to maybe deal with:</b> <br>
What to do with 0 total records? - drop or use some averaging technique?
Do we combine audit analytics dataset with another one (PRC? Gordon said the PRC data is more erroneous than audit analytics, audit analytics more trusted) - make sure no repeats

<b>Resources for WRDS:</b> <br>
WorldScope (I think this is what we want): 
    https://wrds-web.wharton.upenn.edu/wrds//ds/tfn/ws/stock/index.cfm#variablesTab
    https://wrds-web.wharton.upenn.edu/wrds//ds/compd/secm/index.cfm
* Use: 5001 -- Market Price - Year End, stuff like 5015 -- End Market Price - January, 5601 -- ticker, 6038 -- IBES ticker

CRSP also works but I think WorldScope is better.
https://wrds-web.wharton.upenn.edu/wrds//ds/crsp/stock_a/msf.cfm
https://wrds-web.wharton.upenn.edu/wrds/tools/variable.cfm?library_id=20&file_id=123976


<b>There already seems to be a nice data reader for this! https://www.youtube.com/watch?v=57qAxRV577c USe this instead maybe?, stop using other stuff</b><br>

In [2]:
db = wrds.Connection()

Enter your WRDS username [gabri]:gnaval
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [3]:
# Example WRDS code
# db.list_tables('crsp')
db.get_table(library = 'crsp', table ='msf', obs = 10)

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,ret,bid,ask,shrout,cfacpr,cfacshr,altprc,spread,altprcdt,retx
0,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1985-12-31,NaN,NaN,NaN,...,NaN,None,None,NaN,NaN,NaN,-2.562500,NaN,1986-01-07,NaN
1,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-01-31,-2.50000,-4.437500,-4.375000,...,NaN,None,None,3680.0,1.0,1.0,-4.375000,0.25000,1986-01-31,NaN
2,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-02-28,-3.25000,-4.375000,-3.250000,...,-0.257143,None,None,3680.0,1.0,1.0,-3.250000,0.25000,1986-02-28,-0.257143
3,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-03-31,-3.25000,-4.437500,-4.437500,...,0.365385,None,None,3680.0,1.0,1.0,-4.437500,0.12500,1986-03-31,0.365385
4,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-04-30,-4.00000,-4.312500,-4.000000,...,-0.098592,None,None,3793.0,1.0,1.0,-4.000000,0.25000,1986-04-30,-0.098592
5,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-05-30,-3.06250,-4.218750,-3.109375,...,-0.222656,None,None,3793.0,1.0,1.0,-3.109375,0.09375,1986-05-30,-0.222656
6,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-06-30,-2.90625,-3.296875,-3.093750,...,-0.005025,None,None,3793.0,1.0,1.0,-3.093750,0.06250,1986-06-30,-0.005025
7,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-07-31,-2.59375,-3.437500,-2.843750,...,-0.080808,None,None,3793.0,1.0,1.0,-2.843750,0.06250,1986-07-31,-0.080808
8,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-08-29,-1.03125,-2.625000,-1.093750,...,-0.615385,None,None,3793.0,1.0,1.0,-1.093750,0.06250,1986-08-29,-0.615385
9,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-09-30,-0.96875,-1.281250,-1.031250,...,-0.057143,None,None,3793.0,1.0,1.0,-1.031250,0.06250,1986-09-30,-0.057143


In [101]:
# Example WRDS code
# data_5rows = db.get_table(library='wrdssec', table='wrds_nlpsa', columns = ['gvkey', 'cik', 'filename', 'finterms_litigious_count'], obs=5)
pd.set_option('display.max_rows', 10)

sql_query="""
SELECT *
FROM trws.wrds_ws_stock
WHERE ITEM5601 = 'GEO' AND year_=2020
"""
data_query = db.raw_sql(sql_query)
display(data_query)
# data_query[data_query['item6038'].str.contains('CFX') == True]


,code,year_,freq,seq,date_,item5001,item5002,item5003,item5004,item5005,item5015,item5016,item5017,item5020,item5021,item5022,item5025,item5026,item5027,item5030,item5031,item5032,item5035,item5036,item5037,item5040,item5041,item5042,item5045,item5046,item5047,item5050,item5051,item5052,item5055,item5056,item5057,item5060,item5061,item5062,item5065,item5066,item5067,item5070,item5071,item5072,item5085,item5100,item5101,item5102,item5110,item5121,item5122,item5123,item5124,item5125,item5126,item5127,item5128,item5131,item5132,item5133,item5134,item5140,item5141,item5142,item5143,item5144,item5151,item5152,item5153,item5154,item5155,item5156,item5157,item5158,item5171,item5172,item5173,item5174,item5175,item5176,item5177,item5178,item5191,item5192,item5194,item5200,item5201,item5202,item5203,item5210,item5211,item5212,item5213,item5214,item5217,item5218,item5219,item5220,item5221,item5222,item5223,item5224,item5225,item5226,item5227,item5228,item5230,item5240,item5251,item5252,item5253,item5254,item5260,item5261,item5262,item5263,item5264,item5290,item5291,item5292,item5293,item5294,item5301,item5303,item5309,item5326,item5475,item5476,item5486,item5491,item5501,item5502,item5503,item5505,item5506,item5507,item5508,item5509,item5575,item5576,item5601,item5651,item5725,item5901,item5902,item5903,item5904,item5905,item5910,item5911,item5912,item5913,item5917,item5918,item5919,item5920,item5923,item5924,item5925,item5926,item5929,item5930,item5931,item5932,item5935,item5936,item5937,item5938,item5941,item5942,item5943,item5944,item5947,item5948,item5949,item5950,item5953,item5954,item5955,item5956,item5959,item5960,item5961,item5962,item5965,item5966,item5967,item5968,item5970,item5971,item5972,item5973,item5976,item5977,item5978,item5979,item6004,item6006,item6008,item6035,item6038,item6105,item8001,item8002,item8004,item8006,item8010,item8011,item8021,item8801,item8803,item8805,item8806,item9100,item9101,item9104,item9106,item9111,item9116,item9121,item9126,item9200,item9201,item9204,item9206,item9211,item9216,item9221,item9226,item9300,item9301,item9304,item9306,item9311,item9316,item9321,item9326,item9400,item9401,item9404,item9406,item9411,item9416,item9421,item9426,item9504,item9521,item9604,item9621,item9704,item9721,item9904,item10010,item10030,item18100,item18193,item18202,item18205,item18208,item18209,item18210,item18211,item18259,item18260,item18261
0,19875.0,2020.0,A,1.0,2020-10-30,NaN,NaN,NaN,NaN,NaN,0.206,0.240,0.195,0.196,0.212,0.196,0.183,0.201,0.175,0.203,0.218,0.183,0.201,0.216,0.194,0.241,0.246,0.199,0.233,0.250,0.219,0.232,0.246,0.229,0.227,0.236,0.223,0.239,0.250,0.222,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,1,NaN,NaN,None,NaN,-0.000120,-0.000030,-0.000100,NaN,None,None,None,None,-0.00012,-0.00003,-0.00010,NaN,None,NaN,NaN,None,NaN,None,-0.00082,-0.00083,-0.00091,NaN,NaN,-0.00011,-0.00003,-0.00006,NaN,NaN,-0.000120,-0.000030,-0.000100,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,GEO,NaN,None,2020-06-30,2020-08-12,2020-10-29,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6506171,PHY2699D1064,C608H2670,@IGT,C608H2670,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN
1,57330.0,2020.0,A,1.0,2020-10-30,NaN,NaN,NaN,NaN,NaN,1.072,1.228,1.062,0.921,1.158,0.901,0.739,0.946,0.483,0.730,0.760,0.692,0.656,0.740,0.621,0.721,0.910,0.660,0.650,0.770,0.643,0.659,0.723,0.639,0.598,0.668,0.585,0.468,0.645,0.450,

In [15]:
# PRC Dataset 
PRC_df = pd.read_csv("../data/prc.csv" )

# Data Cleaning
PRC_df.drop(PRC_df.columns[[13,14,15]], axis=1, inplace = True)
PRC_df = PRC_df[PRC_df['Total Records'].notna()]

def records_to_int(record_str):
    return int(str(record_str).replace(",", ""))

PRC_df['Total Records'] = PRC_df['Total Records'].apply(records_to_int)

# Drop rows with 0 total records (Maybe toggle this off we want to fill in data)
PRC_df = PRC_df[PRC_df['Total Records'] != 0]

In [27]:
# Audit Analytics Dataset
xls = pd.ExcelFile('../data/audit_analytics.xlsx')
aa_records_df = pd.read_excel(xls, 'PublicCyber')
aa_ddos_df = pd.read_excel(xls, 'DDoS')

In [91]:
test_df = aa_records_df[aa_records_df['Year'] < 2021].head(50)
test_df

,Company name,Related party,Target relationship to Parent,CIK,Ticker,Market,IRS Number,Parent CIK,Parent Name,Bus Street 1,Bus Street 2,City,County,State Code,State Name,Zip,Region,Bus Phone,Inc. State Code,Inc. State Name,SIC Code,SIC Description,NAICS Code,NAICS Description,Year,Month,Breach No.,Registration Number - Auditor Date of Breach,Auditor Date of Breach,Auditor Key Date of Breach,Auditor Affiliate Date of Breach,Registration Number - Auditor Affiliate Date of Breach,Auditor City,Auditor State Code,Auditor State Name,Auditor State Region,Registration Number - Current Auditor,Current Auditor,Current Auditor Key,Current Affiliate Auditor,Registration Number - Current Affiliate Auditor,Auditor City.1,Auditor State Code.1,Auditor State Name.1,Auditor State Region.1,Date of Breach,Date Became Aware of Breach,Date of Disclosure,Type of Breach,Number of Records,Type of Info,Information,Attack,Link,Source,Release,6/8-K SEC notification,Date,First Estimate Filing,Section,Filing Date,Filing Disclosure,Cost/Estimate,Reimbursements,Insurance,Litigation Docket,Lead Defendant Representation,Lead Plaintiff Representation,Litigation Settlement
33,"T-Mobile US, Inc.","T-Mobile US, Inc.",Parent,1283699,TMUS,Nasdaq Global Market,20-0836269,NaN,NaN,8144 WALNUT HILL LANE,SUITE 800,DALLAS,Dallas,TX,TEXAS,75231,US Southwest,(800) 318-9270,DE,DELAWARE,4812.0,Radiotelephone Communications,517212.0,Cellular and Other Wireless Telecommunications,2020,12,851,238.0,PricewaterhouseCoopers LLP,1.0,NaN,NaN,Seattle,WA,WASHINGTON,US West,238.0,PricewaterhouseCoopers LLP,1.0,,NaN,Seattle,WA,WASHINGTON,US West,NaN,NaN,2020-12-30,HACK,200000,Personal,Phone Number | Other,Unauthorized Access,https://www.bleepingcomputer.com/news/security...,Bleeping Computer,T-Mobile has announced a data breach exposing ...,https://www.sec.gov/Archives/edgar/data/128369...,2021-02-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,WHIRLPOOL CORP /DE/,WHIRLPOOL CORP /DE/,Parent,106640,WHR,NYSE,38-1490038,NaN,NaN,WHIRLPOOL CNTR 2000 M 63,C/O CORPORATE SECRETARY,BENTON HARBOR,Berrien,MI,MICHIGAN,49022-2692,US Midwest,(269) 923-5000,DE,DELAWARE,3630.0,Household Appliances,335228.0,Other Major Household Appliance Manufacturing,2020,12,850,42.0,Ernst & Young LLP,2.0,NaN,NaN,Chicago,IL,ILLINOIS,US Midwest,42.0,Ernst & Young LLP,2.0,,NaN,Chicago,IL,ILLINOIS,US Midwest,NaN,NaN,2020-12-28,HACK,NaN,Personal,Other,Ransomware,https://www.bleepingcomputer.com/news/security...,Bleeping Computer,Home appliances giant Whirlpool suffered a ran...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,Colfax CORP,"DJO Global, Inc.",Subsidiary,1420800,CFX,NYSE,54-1887631,NaN,NaN,420 NATIONAL BUSINESS PARKWAY,5TH FLOOR,ANNAPOLIS JUNCTION,Howard,MD,MARYLAND,20701,US Mid Atlantic,(301) 323-9000,DE,DELAWARE,3561.0,Pumps and Pumping Equipment,333911.0,Pump and Pumping Equipment Manufacturing,2020,12,849,42.0,Ernst & Young LLP,2.0,NaN,NaN,Baltimore,MD,MARYLAND,US Mid Atlantic,42.0,Ernst & Young LLP,2.0,,NaN,Baltimore,MD,MARYLAND,US Mid Atlantic,11/8/2020 - 11/9/2020,2020-11-16 00:00:00,2020-12-23,HACK,NaN,Financial,Name|SSN|Other|Bank Account,Unauthorized Access,https://oag.ca.gov/system/files/DJO%20-%20Noti...,Office of Attorney General: California,We write to share important information with y...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,"VMWARE, INC.",SolarWinds Corp,Third-Party,1124610,VMW,NYSE,94-3292913,NaN,NaN,3401 HILLVIEW AVENUE,NaN,PALO ALTO,Santa Clara,CA,CALIFORNIA,94304,US West,(650) 427-5000,DE,DELAWARE,7372.0,Prepackaged Software,541511.0,Custom Computer Programming Services,2020,12,848,238.0,PricewaterhouseCoopers LLP,1.0,NaN,NaN,San Jose,CA,CALIFORNIA,US West,238.0,PricewaterhouseCoopers LLP,1.0,,NaN,San Jose,CA,CALIFORNIA,US West,NaN,NaN,2020-12-21,HACK,NaN,ND,ND,Malware,https://www.wsj.com/articles/solarwinds-hack-v...,Wall Street Journal,The suspected Russian hackers behind breaches ...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,NVIDIA CORP,SolarWinds C

In [97]:
def month_to_stock_col(month):
    return "ITEM" + str(5010 + 5 * month)


def find_initial_stock(row):
    sql_query = """
    SELECT {} AS init_price, ITEM5601 AS ticker, ITEM6038 AS IBES_ticker 
    FROM trws.wrds_ws_stock
    WHERE ITEM5601 ='{}' and year_={}
    """.format(month_to_stock_col(row['Month']), row['Ticker'], row['Year'])
    df = db.raw_sql(sql_query)
#     display(df)
    num_rows = len(df.index)
    if num_rows == 0:
        return np.nan
    elif num_rows == 1:
        return df['init_price'].iloc[0]
    else:
        print(df.columns)
        match_ibes_df = df[df['ibes_ticker'].str.contains(row['Ticker']) == True]
        print("Number of matching IBES tickers: %s" % len(match_ibes_df))
        num_rows = len(match_ibes_df)
        if num_rows == 0:
            print("No matching IBES tickers found")
            display(df)
        elif num_rows == 1:
            return match_ibes_df['init_price'].iloc[0]
        else:
            print("Multiple matching IBES tickers found")
            display(match_ibes_df)
    return np.nan

# sql_query="""
# SELECT *
# FROM trws.wrds_ws_stock
# WHERE year_=2021
# """

test_df['init_price'] = test_df.apply(lambda row: find_initial_stock(row), axis=1)
test_df['init_price']

Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 2
Multiple matching IBES tickers found


,init_price,ticker,ibes_ticker
3,None,CLH,CLHB
4,None,CLH,@CLHH


Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 0
No matching IBES tickers found


,init_price,ticker,ibes_ticker
0,None,SPOT,020K
1,None,SPOT,None


Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 0
No matching IBES tickers found


...

,init_price,ticker,ibes_ticker
0,0.013,TMX,@YAY
1,14500.000,TMX,None
2,47.090,TMX,007X


Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 0
No matching IBES tickers found


,init_price,ticker,ibes_ticker
0,7.69,VOXX,VOX
1,7.69,VOXX,None
2,NaN,VOXX,None


Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1


33          NaN
34          NaN
35          NaN
36          NaN
37          NaN
        ...    
78    760.13000
79     16.84168
80     28.74000
81          NaN
82     14.50000
Name: init_price, Length: 50, dtype: float64

In [30]:
# Ticker - Don't think we need this 
ticker_df = pd.read_csv("../data/ticker.csv" )
ticker_df

,Unnamed: 0,cik_str,ticker,title
0,0,320193,AAPL,Apple Inc.
1,1,789019,MSFT,MICROSOFT CORP
2,2,1018724,AMZN,AMAZON COM INC
3,3,1652044,GOOG,Alphabet Inc.
4,4,1293451,TCEHY,Tencent Holdings Ltd
...,...,...,...,...
11351,11351,1830081,CFVIU,CF Acquisition Corp. VI
11352,11352,1830063,NAACU,North Atlantic Acquisition Corp
11353,11353,1829889,IIIIW,INSU ACQUISITION CORP III
11354,11354,1829999,FOXWU,FoxWayne Enterprises Acquisition Corp.
